# Problèmes

### 1)
Primal: $\underset{x \in \mathbb{R}^3}{\min} \frac{1}{2} x' Q x + c' x ~~~~~~~~ s.t. ~ A x = b ~, ~~ x \ge 0$

with  $Q = \begin{bmatrix} 6 & 2 & 1\\ 2 & 5 & 2\\ 1 & 2 & 4 \end{bmatrix} $, $c = \begin{bmatrix} -8 \\ -3 \\ -3 \end{bmatrix}$, $A = \begin{bmatrix} 1 & 0 &1 \\ 0 & 1 & 1 \end{bmatrix}$ , and $b = \begin{bmatrix} 3 \\ 0 \end{bmatrix} $.


### 2) 
AFIRO        28 rows    32 cols        88 nonzeros       794 bytes      -4.6475314286E+02 optimalvalue

In [5]:
using LinearAlgebra
using LaTeXStrings
using DataFrames
using DataStructures
using QuadraticModels
using Printf
using SparseArrays
using BenchmarkTools
using NLPModels
using LinearOperators
using QPSReader
using SolverTools
using SolverBenchmark

# Mehrotra formulation plus générale:

min $c_0 + c'x + \frac{1}{2} x'Hx ~~~~$ s.t. $ Ax = b $ and $l \le x \le u$ 

In [6]:
function display_results(result)
    # fonction pour l'affichage
    println("\n-----------------------------------------------------------------------")
    println("------------------------------- RESULTS -------------------------------")
    result
end

function init_x0(lvar, uvar)
    # choice of an init point x0
    x0 = zeros(length(lvar))
    for i=1:length(x0)
        if lvar[i] == -Inf && uvar[i] == Inf
            x0[i] = 0.
        elseif lvar[i] == -Inf && uvar[i] != Inf
            x0[i] = uvar[i] - 1.
        elseif lvar[i] != -Inf && uvar[i] == Inf
            x0[i] = lvar[i] + 1.
        else
            x0[i] = (lvar[i] + uvar[i]) / 2 
        end
    end  
    return x0
end

function init_x0_lsq(A, b, lvar, uvar)
    x_tilde = A\b
    n = length(x_tilde)
    for i=1:n
        if x_tilde[i] <= lvar[i]
            x_tilde[i] = lvar[i] + 1
        elseif uvar[i] <= x_tilde[i]
            x_tilde[i] = uvar[i] - 1
        end
        if !(lvar[i] < x_tilde[i] < uvar[i])
            x_tilde[i] = (lvar[i] + uvar[i])/2
        end
    end

    return x_tilde
end
    



init_x0_lsq (generic function with 1 method)

In [64]:
function compute_α_dual(v, dir_v)
    n = length(v)
    if n == 0
        return 1.
    end
    α = 1.
    for i=1:n
        if dir_v[i] < 0.
            α_new = -v[i] * 0.999 / dir_v[i]
            if α_new < α
                α = α_new
            end
        end
    end
    return α
end


    
function compute_α_primal(v, dir_v, lvar, uvar)
    n = length(v)
    α_l, α_u = 1., 1.
    for i=1:n
        if dir_v[i] > 0.
            α_u_new = (uvar[i] - v[i]) * 0.999 / dir_v[i]
            if α_u_new < α_u
                α_u = α_u_new
            end
        elseif dir_v[i] < 0.
            α_l_new = (lvar[i] - v[i]) * 0.999 / dir_v[i]
            if α_l_new < α_l
                α_l = α_l_new
            end
        end
    end
    return min(α_l, α_u)
end

function compute_μ(x_l, x_u, s_l, s_u, lvar, uvar, nb_low, nb_upp)
    #x_l coordinates of x corresponding to finite lower bounds ( resp. finite upper bounds for x_u)
    # arguments must have finite bounds 
    return (s_l' * (x_l-lvar) + s_u' * (uvar-x_u)) / (nb_low + nb_upp)
end


function is_in_Neighborhood_inf(gamma, x_l, x_u, s_l, s_u, lvar, uvar)
    # check if the current point is in N_inf(gamma)
    # true : (xi_l - lvari) * si_l >= gamma mu   and   (uvari - xi_u) * si_u >= gamma mu 
    mu = Compute_mu(x_l, x_u, s_l, s_u, lvar, uvar)
    for i=1:length(x_l)
        if (x_l[i] - lvar[i]) * s_l[i] < gamma*mu
            return false
        end
    end
    for i=1:length(x_u)
        if (uvar[i] - x_u[i]) * s_u[i] < gamma*mu
            return false
        end
    end
    return true
end

is_in_Neighborhood_inf (generic function with 1 method)

In [65]:
function solve_augmented_system_aff(J_fact, rc, rb, x, s_l, s_u, ilow, 
                                    iupp, lvar, uvar, n_cols)
    
    F_x_λ_aff = [-rc 
                 -rb]
    F_x_λ_aff[ilow] += s_l[ilow]
    F_x_λ_aff[iupp] -= s_u[iupp]
    
    Δ_x_λ_aff = J_fact\F_x_λ_aff
    Δ_aff = [Δ_x_λ_aff
             -s_l[ilow] - s_l[ilow].*Δ_x_λ_aff[1:n_cols][ilow]./(x[ilow]-lvar[ilow])
             - s_u[iupp] - s_u[iupp].*Δ_x_λ_aff[1:n_cols][iupp]./(x[iupp]-uvar[iupp])]
    return Δ_aff
end

function solve_augmented_system_cc(J_fact, σ, μ,x, s_l, s_u, ilow, iupp,  lvar, uvar,
                                   Δ_aff, n_cols, n_rows, n_low)
    
    rxs_l = -σ*μ .+ Δ_aff[1:n_cols][ilow].*Δ_aff[n_rows+n_cols+1: n_rows+n_cols+n_low]
    rxs_u = σ*μ .+ Δ_aff[1:n_cols][iupp].*Δ_aff[n_rows+n_cols+n_low+1: end]
    
    F_x_λ_cc = spzeros(n_cols+n_rows,1)
    F_x_λ_cc[ilow] += rxs_l./(x[ilow]-lvar[ilow])
    F_x_λ_cc[iupp] -= rxs_u./(x[iupp]-uvar[iupp])
    
#     F_x_λ_cc = [ I_low*inv_X_L*rxs_l + I_op_upp*inv_X_U*rxs_u
#                                    Or                 ]
    Δ_x_λ_cc = J_fact\F_x_λ_cc
    Δ_cc = [Δ_x_λ_cc
            -(rxs_l+s_l[ilow].*Δ_x_λ_cc[1:n_cols][ilow])./(x[ilow]-lvar[ilow])
            -(rxs_u+s_u[iupp].*Δ_x_λ_cc[1:n_cols][iupp])./(x[iupp]-uvar[iupp]) ]
    return Δ_cc
end


solve_augmented_system_cc (generic function with 1 method)

In [80]:
function mehrotraPCQuadBounds(QM, max_iter; ϵ_pdd=1e-8, ϵ_rb=1e-6, ϵ_rc=1e-6,
                              tol_Δx=1e-16, ϵ_μ=1e-16, display=true)
    
    start_time = time()
    elapsed_time = 0.0
    
    # get variables from QuadraticModel
    lvar, uvar = QM.meta.lvar, QM.meta.uvar
    n_cols = length(lvar)
    Oc = zeros(n_cols)
    ilow, iupp = [QM.meta.ilow; QM.meta.irng], [QM.meta.iupp; QM.meta.irng] # finite bounds index
    n_low, n_upp = length(ilow), length(iupp) # number of finite constraints
    A = jac(QM, Oc)
    n_rows, n_cols = size(A) 
    Q = hess(QM, Oc)
    c = grad(QM, Oc)
    c0 = obj(QM, Oc)
    @assert QM.meta.lcon == QM.meta.ucon # equality constraint (Ax=b)
    b = QM.meta.lcon
    x = init_x0_lsq(A, b, lvar, uvar)
    @assert all(x .> lvar) && all(x .< uvar)
    s_l, s_u = copy(Oc), copy(Oc)
    s_l[ilow] .= 1.
    s_u[iupp] .= 1.
    Qx = Q * x
    λ = sparse(A') \ (c+Qx) # least square initialisation, s_0 = stilde_0
    
    rb = A * x - b
    rc = -Qx + A' * λ + s_l - s_u - c
    μ = compute_μ(x[ilow], x[iupp], 
                  s_l[ilow], s_u[iupp], 
                  lvar[ilow], uvar[iupp],
                  n_low, n_upp)

    k = 0

    # stopping criterion
    #n_c_p1 =  norm(c, Inf) + 1.
    #n_b_p1 =  norm(b, Inf) + 1.
    xTQx_2 = x' * Qx / 2
    cTx = c' * x
    pri_obj = xTQx_2 + cTx + c0
    dual_obj = b' * λ - xTQx_2 + s_l[ilow]'*lvar[ilow] - s_u[iupp]'*uvar[iupp] +c0
    pdd = abs(pri_obj - dual_obj ) / (1 + abs(pri_obj)) 
    max_rc, max_rb = norm(rc, Inf), norm(rb, Inf)
    optimal = pdd < ϵ_pdd && max_rc < ϵ_rc && max_rb < ϵ_rb
    small_Δx, small_μ = false, μ < ϵ_μ
    
    # init regularization values
    ρ = 1.
    J_augm = [-Q       A'
                A     ρ*I ]

    # display
    if display == true
        println("Iter    primal      pdd      rb cond    rc cond    step x       μ")
        @printf("%4d  % 9.2e   %7.1e    %7.1e    %7.1e    %7.1e   %7.1e\n", 
                k, pri_obj, pdd, max_rb, max_rc ,0., μ)
    end
    

    while k<max_iter && !optimal && !small_Δx && !small_μ
        
            
            # Affine scaling direction
        
#         J_augm = [-Q-I_low*inv_X_L*S_low-I_op_upp*inv_X_U*S_upp-ρ*I        A'
#                                A                                         ρ*I  ]
        
        #J_augm[1:n_cols, 1:n_cols] = -Q - ρ*I
        #J_augm[1:n_cols+1:n_cols^2] = -Q[1:n_cols+1:n_cols^2] .- ρ
        J_augm[diagind(J_augm)[1:n_cols]] = -Q[diagind(Q)] .- ρ
        for j in ilow
            J_augm[j,j] -=  s_l[j] / (x[j] - lvar[j]) 
        end
        for j in iupp
            J_augm[j,j] += s_u[j] / (x[j] - uvar[j])
        end

        J_fact = ldlt(Symmetric(J_augm))
        #J_fact = lu(J_augm)
        
        Δ_aff = solve_augmented_system_aff(J_fact, rc, rb, x, s_l, s_u, ilow, 
                                           iupp, lvar, uvar, n_cols)

        α_aff_pri = compute_α_primal(x, Δ_aff[1:n_cols], lvar, uvar)
        α_aff_dual_l = compute_α_dual(s_l[ilow], Δ_aff[n_rows+n_cols+1: n_rows+n_cols+n_low])
        α_aff_dual_u = compute_α_dual(s_u[iupp], Δ_aff[n_rows+n_cols+n_low+1:end])

        # alpha_aff_dual_final is the min of the 2 alpha_aff_dual
        α_aff_dual_final = min(α_aff_dual_l, α_aff_dual_u)
        
        μ_aff = compute_μ(x[ilow] + α_aff_pri * Δ_aff[1:n_cols][ilow],
                          x[iupp] + α_aff_pri * Δ_aff[1:n_cols][iupp],
                          s_l[ilow] + α_aff_dual_final * Δ_aff[n_rows+n_cols+1: n_rows+n_cols+n_low],
                          s_u[iupp] + α_aff_dual_final * Δ_aff[n_rows+n_cols+n_low+1: end],
                          lvar[ilow], uvar[iupp],
                          n_low, n_upp)
        
        σ = (μ_aff / μ)^3

        # corrector and centering step
        
        Δ_cc = solve_augmented_system_cc(J_fact, σ, μ,x, s_l, s_u, ilow, iupp,  lvar, uvar,
                                         Δ_aff, n_cols, n_rows, n_low)
        
        Δ = Δ_aff + Δ_cc # final direction
        
        α_pri = compute_α_primal(x, Δ[1:n_cols], lvar, uvar)
        α_dual_l = compute_α_dual(s_l[ilow], Δ[n_rows+n_cols+1: n_rows+n_cols+n_low])
        α_dual_u = compute_α_dual(s_u[iupp], Δ[n_rows+n_cols+n_low+1: end])

        α_dual_final = min(α_dual_l, α_dual_u)

        # new parameters
        x += α_pri * Δ[1:n_cols]
        λ += α_dual_final * Δ[n_cols+1: n_rows+n_cols]
        s_l[ilow] += α_dual_final * Δ[n_rows+n_cols+1: n_rows+n_cols+n_low]
        s_u[iupp] += α_dual_final * Δ[n_rows+n_cols+n_low+1: end]
        
        n_Δx = α_pri * norm(Δ[1:n_cols])
        μ = compute_μ(x[ilow], x[iupp], 
                      s_l[ilow], s_u[iupp], 
                      lvar[ilow], uvar[iupp],
                      n_low, n_upp)
        Qx = Q * x
        xTQx_2 = x' * Qx / 2
        cTx = c' * x
        pri_obj = xTQx_2 + cTx + c0
        dual_obj = b' * λ - xTQx_2 + s_l[ilow]'*lvar[ilow] - s_u[iupp]'*uvar[iupp] +c0

        rb = A * x - b
        rc = -Qx + A' * λ + s_l - s_u - c
        
        # update stopping criterion values:
        
        pdd = abs(pri_obj - dual_obj ) / (1 + abs(pri_obj)) 
        max_rc, max_rb = norm(rc, Inf), norm(rb, Inf)
        optimal = pdd < ϵ_pdd && max_rb < ϵ_rb && max_rc < ϵ_rc
        small_Δx, small_μ = n_Δx < tol_Δx, μ < ϵ_μ
        k += 1
        if ρ >= 1e-7
            for j in n_cols+1:n_rows+n_cols
                J_augm[j,j] /= 10.
            end
            ρ /= 10.
        end
        
        if display == true
            @printf("%4d  % 9.2e   %7.1e    %7.1e    %7.1e    %7.1e   %7.1e\n", 
                    k, pri_obj, pdd, max_rb, max_rc,n_Δx , μ)
        end
        

    end
    
    if display == true
        criteria = [k >= max_iter,  optimal, small_Δx, small_μ]
        criteria_names = ["reached max_iter",  "optimal", 
            "n_Δx <= small_Δx", "μ <= ϵ_μ"]
        println("\n stopping criterion = ",criteria_names[findall(criteria)])
    end
    
    elapsed_time = time() - start_time
    
    if k>= max_iter
        status = :max_iter
    else
        status = :acceptable
    end
    
    stats = GenericExecutionStats(status, QM, solution = x,
                                  objective = pri_obj , 
                                  dual_feas = max_rc, 
                                  primal_feas = max_rb,
                                  multipliers = λ,
                                  multipliers_L = s_l,
                                  multipliers_U = s_u,
                                  iter = k, elapsed_time=elapsed_time)
    return stats
end


mehrotraPCQuadBounds (generic function with 1 method)

### bibliothèque QuadraticModels

optimize $c_0 + c'x + \frac{1}{2} x'Hx ~~~~$ s.t. $~~L \le Ax \le U$ and $l \le x \le u$ 

Ici L = lcon, U = ucon, l = lvar, u = uvar

In [81]:
# probleme1
Q = [6 2 1
    2 5 2
    1 2 4]
c = [-8; -3; -3]
c0 = 0.
A = [1 0 1
    0 1 1]
b = [0; 3]
lvar = [0;0;0]
uvar = [Inf; Inf; Inf]
lcon = b
ucon = b

x01 = [1.; 2.; 3.];


In [82]:
QM = QuadraticModel(c, Q, A=A, lcon=lcon, ucon=ucon, lvar=lvar, uvar=uvar, x0=x01, c0=c0, name="QM1")
SM = SlackModel(QM)
typeof(SM)

QuadraticModel

In [83]:
stats_mpc1 =  mehrotraPCQuadBounds(SM, 20)
println(stats_mpc1)

Iter    primal      pdd      rb cond    rc cond    step x       μ
   0   1.40e+01   6.7e-01    2.0e+00    2.3e+00    0.0e+00   1.7e+00
   1   9.35e+00   1.9e-01    1.5e+00    8.7e-01    5.1e-01   5.3e-02
   2   1.05e+01   4.9e-01    3.1e-01    1.2e+00    8.3e-01   3.0e-03
   3   1.19e+01   2.2e-01    1.5e-01    6.0e-01    1.7e-01   4.3e-04
   4   1.35e+01   2.8e-04    1.6e-04    2.8e-01    2.1e-01   8.5e-06
   5   1.35e+01   7.8e-05    2.3e-05    6.4e-04    3.1e-04   9.0e-07
   6   1.35e+01   1.3e-06    1.1e-07    4.3e-05    2.3e-05   4.0e-09
   7   1.35e+01   6.1e-09    1.0e-10    2.5e-07    1.1e-07   4.0e-12

 stopping criterion = ["optimal"]
Generic Execution stats
  status: solved to within acceptable tolerances
  objective value: 13.499999998714355
  primal feasibility: 1.0205747358327244e-10
  dual feasibility: 2.457455936522024e-7
  solution: [1.1764789108368788e-11  2.999999999897765  1.7747632420055182e-13]
  multipliers: [-9.012925868118199  12.000000028759404]
  multipliers_

In [84]:
#pb verif

QM_verif = QuadraticModel(c*(-1), Q, A=A*(-1), lcon=lcon, ucon=ucon, lvar=-uvar, uvar=lvar, x0=x01, c0=c0, name="QM1")
SM_verif = SlackModel(QM_verif)
stats_mpc_verif =  mehrotraPCQuadBounds(SM_verif, 20);
println(stats_mpc_verif)

Iter    primal      pdd      rb cond    rc cond    step x       μ
   0   1.40e+01   6.7e-01    2.0e+00    2.3e+00    0.0e+00   1.7e+00
   1   9.35e+00   1.9e-01    1.5e+00    8.7e-01    5.1e-01   5.3e-02
   2   1.05e+01   4.9e-01    3.1e-01    1.2e+00    8.3e-01   3.0e-03
   3   1.19e+01   2.2e-01    1.5e-01    6.0e-01    1.7e-01   4.3e-04
   4   1.35e+01   2.8e-04    1.6e-04    2.8e-01    2.1e-01   8.5e-06
   5   1.35e+01   7.8e-05    2.3e-05    6.4e-04    3.1e-04   9.0e-07
   6   1.35e+01   1.3e-06    1.1e-07    4.3e-05    2.3e-05   4.0e-09
   7   1.35e+01   6.1e-09    1.0e-10    2.5e-07    1.1e-07   4.0e-12

 stopping criterion = ["optimal"]
Generic Execution stats
  status: solved to within acceptable tolerances
  objective value: 13.499999998714355
  primal feasibility: 1.0205747358327244e-10
  dual feasibility: 2.457455936522024e-7
  solution: [-1.1764789108368788e-11  -2.999999999897765  -1.7747632420055182e-13]
  multipliers: [-9.012925868118199  12.000000028759404]
  multiplie

In [85]:
### pb verif 2
c2 = [1; 0; 0]
A2 = Matrix([1, 1, 1]')
b2 = [1];
QM_verif2 = QuadraticModel(c2, zeros(3,3), A=A2, lcon=b2, ucon=b2, lvar=[0.;0;0], 
                uvar=[Inf; Inf; Inf], x0=x01, c0=c0, name="QMverfi2")
SM_verif2 = SlackModel(QM_verif2)
stats_mpc_verif2 =  mehrotraPCQuadBounds(SM_verif2, 20);
println(stats_mpc_verif2)

Iter    primal      pdd      rb cond    rc cond    step x       μ
   0   1.00e+00   3.3e-01    2.0e+00    1.3e+00    0.0e+00   1.0e+00
   1   2.25e-01   4.8e-01    6.9e-01    7.7e-01    8.6e-01   9.0e-02
   2   9.86e-02   1.4e-01    3.1e-02    3.0e-02    4.4e-01   4.0e-02
   3   9.86e-05   2.1e-04    5.9e-04    9.9e-04    1.3e-01   1.5e-04
   4   9.86e-08   5.5e-07    2.9e-07    1.1e-06    5.0e-04   1.5e-07
   5   9.86e-11   5.7e-10    2.2e-10    1.1e-09    2.9e-07   1.5e-10

 stopping criterion = ["optimal"]
Generic Execution stats
  status: solved to within acceptable tolerances
  objective value: 9.861953161759842e-11
  primal feasibility: 2.217128702852733e-10
  dual feasibility: 1.0954280815766992e-9
  solution: [9.861953161759842e-11  0.4999999998398338  0.4999999998398338]
  multipliers: [6.693802245214811e-10]
  multipliers_L: [0.9999999982351917  3.4010421147804937e-10  3.4010421147804937e-10]
  multipliers_U: [0.0  0.0  0.0]
  iterations: 5
  elapsed time: 0.02199983596801757

# Lecture des donnees .SIF

In [86]:

function createQuadraticModel(qpdata)
    return QuadraticModel(qpdata.c, qpdata.qrows, qpdata.qcols, qpdata.qvals,
            Arows=qpdata.arows, Acols=qpdata.acols, Avals=qpdata.avals, 
            lcon=qpdata.lcon, ucon=qpdata.ucon, lvar=qpdata.lvar, uvar=qpdata.uvar,
            c0=qpdata.c0)
end

function displayQuadraticModel(QM)
    #println("A = ", Matrix(jac(QM, QM.meta.x0)))
    #println("Q = ", Matrix(hess(QM, QM.meta.x0)))
    println("lvar = ", QM.meta.lvar)
    println("uvar = ", QM.meta.uvar)
    println("x0 = ", QM.meta.x0)
    #println("lcon = ", QM.meta.lcon)
    #println("ucon = ", QM.meta.ucon)
end

displayQuadraticModel (generic function with 1 method)

# Benchmark

In [87]:
path_pb = "C:\\Users\\Geoffroy Leconte\\Documents\\cours\\TFE\\code\\problemes_netlib"
afiro = string(path_pb, "\\AFIRO.SIF")

"C:\\Users\\Geoffroy Leconte\\Documents\\cours\\TFE\\code\\problemes_netlib\\AFIRO.SIF"

In [88]:
qpdata2 = readqps(afiro)  # obj   -4.6475314286E+02
qpdata2.contypes
QM2 = createQuadraticModel(qpdata2)
SM2 = SlackModel(QM2);
SM2.meta

┌ Info: Using 'AFIRO' as NAME (l. 5)
└ @ QPSReader C:\Users\Geoffroy Leconte\.julia\packages\QPSReader\usDQr\src\readqps.jl:751
┌ Info: Using 'COST' as objective (l. 47)
└ @ QPSReader C:\Users\Geoffroy Leconte\.julia\packages\QPSReader\usDQr\src\readqps.jl:350
┌ Info: Using 'B' as RHS (l. 96)
└ @ QPSReader C:\Users\Geoffroy Leconte\.julia\packages\QPSReader\usDQr\src\readqps.jl:470


  Problem name: Generic-slack
   All variables: ████████████████████ 51     All constraints: ████████████████████ 27    
            free: ⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅ 0                 free: ⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅ 0     
           lower: █████████████⋅⋅⋅⋅⋅⋅⋅ 32               lower: ⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅ 0     
           upper: ████████⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅ 19               upper: ⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅ 0     
         low/upp: ⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅ 0              low/upp: ⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅ 0     
           fixed: ⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅ 0                fixed: ████████████████████ 27    
          infeas: ⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅ 0               infeas: ⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅ 0     
            nnzh: (100.00% sparsity)   0               linear: ████████████████████ 27    
                                                    nonlinear: ⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅ 0     
                                                         nnzj: ( 92.59% sparsity)   102   



In [89]:
stats_mpc2 =  mehrotraPCQuadBounds(SM2, 100)
println(stats_mpc2)

Iter    primal      pdd      rb cond    rc cond    step x       μ
   0   4.38e+02   4.7e+00    1.9e+01    5.9e+00    0.0e+00   3.6e+01
   1   3.73e+02   9.0e-01    6.8e+00    8.4e+00    9.9e+00   4.5e+00
   2   4.94e+01   2.2e+00    5.2e-01    5.9e+00    4.8e+01   2.3e+00
   3   3.93e+02   8.1e-01    4.9e-01    5.2e+00    4.9e+01   3.9e+00
   4   3.10e+02   7.7e-01    4.8e-01    5.2e+00    1.4e+01   3.8e+00
   5   3.12e+02   7.7e-01    4.8e-01    5.2e+00    1.7e+00   3.8e+00
   6  -2.54e+00   1.5e-01    3.7e-01    3.0e+00    4.8e+01   2.9e+00
   7  -5.75e+00   8.6e-01    2.1e-06    1.1e+00    1.0e+01   1.1e+00
   8  -1.98e+02   1.0e+00    3.9e-07    9.7e-01    4.9e+02   3.6e+00
   9  -4.16e+02   1.0e+00    3.3e-07    6.4e-01    5.1e+02   9.7e-01
  10  -8.18e+01   1.0e+00    3.0e-07    6.3e-01    8.1e+02   1.0e+00
  11  -1.74e+02   1.0e+00    3.0e-07    6.3e-01    3.2e+02   9.2e-01
  12  -4.23e+02   3.6e+00    3.0e-07    2.5e-01    6.0e+02   3.4e+01
  13  -4.37e+02   8.6e-01    5.1e-08 

In [90]:
@benchmark mehrotraPCQuadBounds(SM2, 100, display=false)

BenchmarkTools.Trial: 
  memory estimate:  2.34 MiB
  allocs estimate:  15939
  --------------
  minimum time:     2.138 ms (0.00% GC)
  median time:      2.206 ms (0.00% GC)
  mean time:        2.480 ms (5.31% GC)
  maximum time:     6.797 ms (40.17% GC)
  --------------
  samples:          2014
  evals/sample:     1

In [91]:
# problem 3   kb2    obj  -1.7499001299E+03
kb2 = string(path_pb, "\\KB2.SIF")
qpdata3 = readqps(kb2)
QM3 = createQuadraticModel(qpdata3)
SM3 = SlackModel(QM3)
SM3.meta

┌ Info: Using 'KB2' as NAME (l. 5)
└ @ QPSReader C:\Users\Geoffroy Leconte\.julia\packages\QPSReader\usDQr\src\readqps.jl:751
┌ Info: Using 'FAT7..J.' as objective (l. 20)
└ @ QPSReader C:\Users\Geoffroy Leconte\.julia\packages\QPSReader\usDQr\src\readqps.jl:350
┌ Info: Using '77BOUND' as BOUNDS (l. 227)
└ @ QPSReader C:\Users\Geoffroy Leconte\.julia\packages\QPSReader\usDQr\src\readqps.jl:622


  Problem name: Generic-slack
   All variables: ████████████████████ 68     All constraints: ████████████████████ 43    
            free: ⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅ 0                 free: ⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅ 0     
           lower: ██████████████⋅⋅⋅⋅⋅⋅ 47               lower: ⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅ 0     
           upper: ████⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅ 12               upper: ⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅ 0     
         low/upp: ███⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅ 9              low/upp: ⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅ 0     
           fixed: ⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅ 0                fixed: ████████████████████ 43    
          infeas: ⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅ 0               infeas: ⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅ 0     
            nnzh: (100.00% sparsity)   0               linear: ████████████████████ 43    
                                                    nonlinear: ⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅ 0     
                                                         nnzj: ( 89.30% sparsity)   313   



In [92]:
stats_mpc3 =  mehrotraPCQuadBounds(SM3, 100);
println(stats_mpc3)

Iter    primal      pdd      rb cond    rc cond    step x       μ
   0   1.17e+01   3.4e+01    6.9e+02    6.1e+00    0.0e+00   6.2e+00
   1  -2.54e+00   5.7e+00    2.6e+02    2.8e+00    4.6e+00   1.2e+00
   2  -3.99e+00   1.5e-01    9.1e+01    4.2e-01    3.9e+00   3.4e-01
   3  -7.45e+00   5.7e-01    3.7e+01    3.4e-01    2.4e+01   2.0e-01
   4  -1.65e+01   8.2e-01    1.1e+01    2.6e-01    5.6e+01   2.0e-01
   5  -1.02e+02   9.7e-01    7.7e-03    1.9e-01    4.3e+02   3.8e-01
   6  -2.25e+02   9.7e-01    7.3e-03    1.9e-01    6.2e+02   7.7e-01
   7  -3.99e+02   9.7e-01    7.1e-03    1.7e-01    8.7e+02   7.0e-01
   8  -6.34e+02   9.6e-01    7.1e-03    1.7e-01    1.2e+03   1.0e+00
   9  -1.26e+03   9.8e-01    6.9e-03    1.6e-01    3.2e+03   1.7e+00
  10  -1.11e+03   9.2e-01    6.8e-03    1.6e-01    7.8e+02   2.2e+00
  11  -1.05e+03   8.7e-01    6.8e-03    1.6e-01    3.0e+02   2.7e+00
  12  -1.07e+03   8.5e-01    6.6e-03    1.6e-01    3.3e+02   3.1e+00
  13  -1.15e+03   8.0e-01    6.5e-03 

In [93]:
@benchmark mehrotraPCQuadBounds(SM3, 100, display = false)

BenchmarkTools.Trial: 
  memory estimate:  6.22 MiB
  allocs estimate:  31480
  --------------
  minimum time:     5.236 ms (0.00% GC)
  median time:      5.338 ms (0.00% GC)
  mean time:        5.859 ms (4.77% GC)
  maximum time:     9.727 ms (30.53% GC)
  --------------
  samples:          853
  evals/sample:     1

In [152]:
# problem 4 SC50A  obj  -6.4575077059E+01
SC50A = string(path_pb, "\\SC50A.SIF")
qpdata4 = readqps(SC50A)
QM4 = createQuadraticModel(qpdata4)
SM4 = SlackModel(QM4)
SM4.meta

┌ Info: Using 'SC50A' as NAME (l. 5)
└ @ QPSReader C:\Users\Geoffroy Leconte\.julia\packages\QPSReader\usDQr\src\readqps.jl:751
┌ Info: Using 'MAXIM' as objective (l. 20)
└ @ QPSReader C:\Users\Geoffroy Leconte\.julia\packages\QPSReader\usDQr\src\readqps.jl:350
┌ Info: Using 'CONST' as RHS (l. 153)
└ @ QPSReader C:\Users\Geoffroy Leconte\.julia\packages\QPSReader\usDQr\src\readqps.jl:470


  Problem name: Generic-slack
   All variables: ████████████████████ 78     All constraints: ████████████████████ 50    
            free: ⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅ 0                 free: ⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅ 0     
           lower: █████████████⋅⋅⋅⋅⋅⋅⋅ 48               lower: ⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅ 0     
           upper: ████████⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅ 30               upper: ⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅ 0     
         low/upp: ⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅ 0              low/upp: ⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅ 0     
           fixed: ⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅ 0                fixed: ████████████████████ 50    
          infeas: ⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅ 0               infeas: ⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅ 0     
            nnzh: (100.00% sparsity)   0               linear: ████████████████████ 50    
                                                    nonlinear: ⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅ 0     
                                                         nnzj: ( 95.90% sparsity)   160   



In [153]:
stats_mpc4 =  mehrotraPCQuadBounds(SM4, 20);
println(stats_mpc4)

Iter    primal      pdd      rb cond    rc cond    step x       μ
   0  -1.00e+00   7.5e+02    1.3e+01    8.8e+00    0.0e+00   2.0e+01
   1  -9.08e-01   1.8e+00    8.2e-01    8.0e-01    3.6e+00   3.0e-01
   2  -2.78e-01   5.8e-02    4.0e-02    2.8e-01    5.2e+00   3.7e-02
   3  -6.07e-01   3.1e-01    7.5e-03    7.6e-02    3.4e+00   8.7e-03
   4  -6.16e+00   8.5e-01    3.9e-03    5.4e-02    7.2e+01   3.9e-02
   5  -4.21e+01   9.8e-01    2.2e-04    3.8e-02    4.7e+02   9.4e-02
   6  -3.01e+01   9.0e-01    2.2e-04    3.8e-02    1.6e+02   9.0e-02
   7  -3.55e+01   9.3e-01    2.2e-04    3.7e-02    1.7e+02   1.0e-01
   8  -3.48e+01   9.0e-01    2.1e-04    3.7e-02    1.2e+02   1.1e-01
   9  -3.87e+01   8.9e-01    2.1e-04    3.6e-02    6.7e+01   1.1e-01
  10  -4.46e+01   8.9e-01    2.1e-04    3.5e-02    1.0e+02   1.2e-01
  11  -5.29e+01   6.9e-01    2.1e-04    3.2e-02    1.7e+02   2.3e-01
  12  -5.22e+01   6.6e-01    2.0e-04    3.2e-02    1.2e+02   2.5e-01
  13  -4.95e+01   1.9e-01    1.8e-04 

# Tests sur tous les problèmes

In [58]:
df_test = DataFrame(id = Int[],
         name = String[],
         status = Symbol[],
         objective = Float64[],
         time = Float64[],
         iter = Int[],
         primal_feas = Float64[],
         dual_feas = Float64[])



i = 0
i_stop = 11   #number of problems to solve
for file_name in readdir(path_pb)
    if file_name[end-3:end] == ".SIF" && !(file_name in ["80BAU3B.SIF" ; "BORE3D.SIF"; 
                                                        "CAPRI.SIF"; "CZPROB.SIF"; 
                                                        "ETAMACRO.SIF"; "FINNIS.SIF";
                                                        "FORPLAN.SIF"; "GREENBEA.SIF";
                                                        "GREENBEB.SIF"; "MAROS.SIF";
                                                        "NESM.SIF"; "PEROLD.SIF";
                                                         "PILOT-JA.SIF"; "PILOT-WE.SIF";
                                                         "PILOT.SIF"; "PILOT4.SIF";
                                                         "PILOT87.SIF"; "PILOTNOV.SIF";
                                                          "RECIPELP.SIF"; "SHELL.SIF";
                                                         "SIERRA.SIF"; "STAIR.SIF";
                                                         "STANDATA.SIF"; "STANDGUB.SIF";
                                                        "STANDMPS.SIF"; "TUFF.SIF";
                                                        "VTP-BASE.SIF"])   &&# problems with fixed variables
                                        !(file_name in ["25FV47.SIF"; "BNL1.SIF";
                                                        "BRANDY.SIF"; "CRE-A.SIF"]) # singular error
                            
        
        i += 1
        println(file_name)
        pb_i = string(path_pb, "\\", file_name)
        if file_name in ["BLEND.SIF"; "DFL001.SIF"; "FORPLAN.SIF"; "GFRD-PNC.SIF"; "SIERRA.SIF"]
            qpdata_i = readqps(pb_i, mpsformat=:fixed) 
        else
            qpdata_i = readqps(pb_i) 
        end

        SM_i = SlackModel(createQuadraticModel(qpdata_i))

        stats_mpc_i =  mehrotraPCQuadBounds(SM_i, 40, display=false)
        
        # 
        push!(df_test, Dict(:id => i,
                             :name => file_name ,
                             :status => stats_mpc_i.status,
                             :objective => stats_mpc_i.objective,
                             :time => stats_mpc_i.elapsed_time,
                             :iter => stats_mpc_i.iter,
                             :primal_feas => stats_mpc_i.primal_feas,
                             :dual_feas => stats_mpc_i.dual_feas))
    end
    
    if i == i_stop
        break
        break
    end
end

markdown_table(stdout, df_test)

ADLITTLE.SIF


┌ Info: Using 'ADLITTLE' as NAME (l. 5)
└ @ QPSReader C:\Users\Geoffroy Leconte\.julia\packages\QPSReader\usDQr\src\readqps.jl:751
┌ Info: Using '.Z....' as objective (l. 20)
└ @ QPSReader C:\Users\Geoffroy Leconte\.julia\packages\QPSReader\usDQr\src\readqps.jl:350


AFIRO.SIF
AGG.SIF


┌ Info: Using 'ZZZZ0001' as RHS (l. 333)
└ @ QPSReader C:\Users\Geoffroy Leconte\.julia\packages\QPSReader\usDQr\src\readqps.jl:470
┌ Info: Using 'AFIRO' as NAME (l. 5)
└ @ QPSReader C:\Users\Geoffroy Leconte\.julia\packages\QPSReader\usDQr\src\readqps.jl:751
┌ Info: Using 'COST' as objective (l. 47)
└ @ QPSReader C:\Users\Geoffroy Leconte\.julia\packages\QPSReader\usDQr\src\readqps.jl:350
┌ Info: Using 'B' as RHS (l. 96)
└ @ QPSReader C:\Users\Geoffroy Leconte\.julia\packages\QPSReader\usDQr\src\readqps.jl:470
┌ Info: Using 'AGG' as NAME (l. 5)
└ @ QPSReader C:\Users\Geoffroy Leconte\.julia\packages\QPSReader\usDQr\src\readqps.jl:751
┌ Info: Using 'OBJECTIV' as objective (l. 509)
└ @ QPSReader C:\Users\Geoffroy Leconte\.julia\packages\QPSReader\usDQr\src\readqps.jl:350
┌ Info: Using 'AGG' as RHS (l. 1852)
└ @ QPSReader C:\Users\Geoffroy Leconte\.julia\packages\QPSReader\usDQr\src\readqps.jl:470


AGG2.SIF


...

┌ Info: Using 'RHS' as RHS (l. 11294)
└ @ QPSReader C:\Users\Geoffroy Leconte\.julia\packages\QPSReader\usDQr\src\readqps.jl:470
┌ Info: Using 'BOEING1' as NAME (l. 5)
└ @ QPSReader C:\Users\Geoffroy Leconte\.julia\packages\QPSReader\usDQr\src\readqps.jl:751
┌ Info: Using 'OBJECTIV' as objective (l. 23)
└ @ QPSReader C:\Users\Geoffroy Leconte\.julia\packages\QPSReader\usDQr\src\readqps.jl:350
┌ Info: Using 'RHS1' as RHS (l. 2372)
└ @ QPSReader C:\Users\Geoffroy Leconte\.julia\packages\QPSReader\usDQr\src\readqps.jl:470
┌ Info: Using 'RANGE1' as RANGES (l. 2446)
└ @ QPSReader C:\Users\Geoffroy Leconte\.julia\packages\QPSReader\usDQr\src\readqps.jl:552


BOEING2.SIF


┌ Info: Using 'INTBOU' as BOUNDS (l. 2492)
└ @ QPSReader C:\Users\Geoffroy Leconte\.julia\packages\QPSReader\usDQr\src\readqps.jl:622
┌ Info: Using 'BOEING2' as NAME (l. 5)
└ @ QPSReader C:\Users\Geoffroy Leconte\.julia\packages\QPSReader\usDQr\src\readqps.jl:751
┌ Info: Using 'OBJECTIV' as objective (l. 23)
└ @ QPSReader C:\Users\Geoffroy Leconte\.julia\packages\QPSReader\usDQr\src\readqps.jl:350
┌ Info: Using 'RHS1' as RHS (l. 898)
└ @ QPSReader C:\Users\Geoffroy Leconte\.julia\packages\QPSReader\usDQr\src\readqps.jl:470
┌ Info: Using 'RANGE1' as RANGES (l. 919)
└ @ QPSReader C:\Users\Geoffroy Leconte\.julia\packages\QPSReader\usDQr\src\readqps.jl:552


|    id |         name |     status | objective |     time |  iter | primal_fe ⋯
|-------|--------------|------------|-----------|----------|-------|---------- ⋯
|     1 | ADLITTLE.SIF | acceptable |   2.3e+05 |  8.2e-02 |    38 |     9.8e- ⋯
|     2 |    AFIRO.SIF | acceptable |  -4.6e+02 |  8.0e-03 |    10 |     8.3e- ⋯
|     3 |      AGG.SIF |   max_iter |   1.9e+09 |  6.0e-01 |    40 |     3.2e+ ⋯
|     4 |     AGG2.SIF |   max_iter |  -4.3e+06 |  1.2e+00 |    40 |     4.9e+ ⋯
|     5 |     AGG3.SIF |   max_iter |   1.7e+09 |  1.2e+00 |    40 |     1.6e+ ⋯
|     6 |    BANDM.SIF | acceptable |  -1.6e+02 |  2.9e-01 |    21 |     3.1e- ⋯
|     7 | BEACONFD.SIF | acceptable |   3.4e+04 |  1.5e-01 |    10 |     7.3e- ⋯
|     8 |    BLEND.SIF | acceptable |  -3.1e+01 |  3.0e-02 |    11 |     4.6e- ⋯
|     9 |     BNL2.SIF |   max_iter |   1.8e+03 |  1.4e+01 |    40 |     1.2e- ⋯
|    10 |  BOEING1.SIF |   max_iter |  -3.3e+02 |  1.1e+00 |    40 |     1.4e- ⋯
|    11 |  BOEING2.SIF | acc

┌ Info: Using 'INTBOU' as BOUNDS (l. 930)
└ @ QPSReader C:\Users\Geoffroy Leconte\.julia\packages\QPSReader\usDQr\src\readqps.jl:622


In [14]:
T = typeof(SM2.meta)
for (name, typ) in zip(fieldnames(T), T.types)
    println("type of the fieldname $name is $typ")
end

type of the fieldname nvar is Int64
type of the fieldname x0 is Array{T,1} where T
type of the fieldname lvar is Array{T,1} where T
type of the fieldname uvar is Array{T,1} where T
type of the fieldname ifix is Array{Int64,1}
type of the fieldname ilow is Array{Int64,1}
type of the fieldname iupp is Array{Int64,1}
type of the fieldname irng is Array{Int64,1}
type of the fieldname ifree is Array{Int64,1}
type of the fieldname iinf is Array{Int64,1}
type of the fieldname nbv is Int64
type of the fieldname niv is Int64
type of the fieldname nlvb is Int64
type of the fieldname nlvo is Int64
type of the fieldname nlvc is Int64
type of the fieldname nlvbi is Int64
type of the fieldname nlvci is Int64
type of the fieldname nlvoi is Int64
type of the fieldname nwv is Int64
type of the fieldname ncon is Int64
type of the fieldname y0 is Array{T,1} where T
type of the fieldname lcon is Array{T,1} where T
type of the fieldname ucon is Array{T,1} where T
type of the fieldname jfix is Array{Int64,1

In [90]:
# problems: AGG.SIF
qpdata5 = readqps(string(path_pb, "\\AGG.SIF"))
QM5 = createQuadraticModel(qpdata5)
SM5 = SlackModel(QM5);

┌ Info: Using 'AGG' as NAME (l. 5)
└ @ QPSReader C:\Users\Geoffroy Leconte\.julia\packages\QPSReader\usDQr\src\readqps.jl:751
┌ Info: Using 'OBJECTIV' as objective (l. 509)
└ @ QPSReader C:\Users\Geoffroy Leconte\.julia\packages\QPSReader\usDQr\src\readqps.jl:350
┌ Info: Using 'AGG' as RHS (l. 1852)
└ @ QPSReader C:\Users\Geoffroy Leconte\.julia\packages\QPSReader\usDQr\src\readqps.jl:470


In [91]:
stats_mpc_5 =  mehrotraPCQuadBounds(SM5, 150)
println(stats_mpc_5)

Iter    primal      pdd      rb cond    rc cond    step x       μ
   0   2.03e+03   1.8e+04    3.2e+06    5.3e+02    0.0e+00   7.4e+04
   1   1.96e+03   6.2e+05    1.8e+06    2.4e+02    3.9e+01   7.4e+04
   2   1.83e+03   5.2e+05    1.8e+06    2.4e+02    1.7e+01   7.4e+04
   3   2.06e+03   4.2e+05    1.8e+06    2.4e+02    1.0e+01   7.4e+04
   4   1.98e+03   4.1e+05    1.8e+06    2.4e+02    3.5e+01   7.4e+04
   5   1.77e+03   4.4e+05    1.8e+06    2.4e+02    6.3e+01   7.4e+04
   6   1.54e+03   4.6e+05    1.8e+06    2.4e+02    5.5e+01   7.4e+04
   7   1.41e+03   5.0e+05    1.8e+06    2.4e+02    1.2e+01   7.4e+04
   8   6.57e+02   1.0e+06    1.8e+06    2.4e+02    1.1e+02   7.4e+04
   9  -2.01e+03   3.7e+05    1.8e+06    2.4e+02    4.1e+02   7.4e+04
  10  -6.96e+03   7.3e+04    1.8e+06    2.4e+02    8.6e+02   7.5e+04
  11  -3.23e+04   2.4e+04    1.8e+06    2.4e+02    3.9e+03   7.7e+04
  12  -4.28e+04   7.6e+03    1.8e+06    2.4e+02    2.0e+03   7.8e+04
  13  -6.77e+04   8.0e+03    1.8e+06 

In [159]:
SM5.meta


  Problem name: Generic-slack
   All variables: ████████████████████ 615    All constraints: ████████████████████ 488   
            free: ⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅ 0                 free: ⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅ 0     
           lower: ███████⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅ 210              lower: ⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅ 0     
           upper: ██████████████⋅⋅⋅⋅⋅⋅ 405              upper: ⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅ 0     
         low/upp: ⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅ 0              low/upp: ⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅ 0     
           fixed: ⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅ 0                fixed: ████████████████████ 488   
          infeas: ⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅ 0               infeas: ⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅ 0     
            nnzh: (100.00% sparsity)   0               linear: ████████████████████ 488   
                                                    nonlinear: ⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅ 0     
                                                         nnzj: ( 99.05% sparsity)   2862  



In [17]:
SM2.meta.iupp

19-element Array{Int64,1}:
 33
 34
 35
 36
 37
 38
 39
 40
 41
 42
 43
 44
 45
 46
 47
 48
 49
 50
 51